# Introduction

Problems from leetcode

### Autocomplete

In [60]:
from heapq import heappop, heappush
from collections import defaultdict

class AutocompleteSystem:

    def __init__(self, sentences, times):
        self.cache = defaultdict(int)
        for s,t in zip(sentences, times):
            self.cache[s] = t
            self.past_prefix = ''

    def inp(self, c):
        """
        
        BRUTE FORCE:
        - select top k from list that match c
        
        Search: Time: O(n*logk), Space: O(k)
        Insert: Time: O(1)
        
        Ex: s = 'i love'
        
        BETTER:
        - use an ordered dict?
        - that way I can stop after the first three hits?
        - (but are ordered dicts just a hash?)
        
        - searching ordered dict --> O(n)  
        - inserting to dict -->  O(log n)
        
        
        TO DO / CONSIDER:
        - equal frequency sentences
        - no matches 
        
        """
                
        #Do the autocompletion
        if c != '#':
        
            prefix = self.past_prefix + c
            self.past_prefix = prefix

            #Return top 3 candidates that match the prompt
            l = len(prefix)
            heap = []
            for sentence, time in self.cache.items():
                if sentence[:l] == prefix:
                    if len(heap) < 3:
                        heappush(heap,(time,sentence))
                    else:
                        #keep equal matches
                        if time == heap[0][0]:
                            heappush(heap,(time,sentence))
                        elif time > heap[0][0]: 
                            heappop(heap)
                            heappush(heap,(time,sentence))
            return heap

        #Add new sentence to cache
        else:
            new_sentence = self.past_prefix
            self.cache[new_sentence] += 1
            self.past_prefix = ''
            return []

#Set up
sentences = ['i love you', 'island','ironman','i love leetcode']
times = [5,3,2,2]
obj = AutocompleteSystem(sentences, times)

#Start the prompt
obj.inp('i')
#obj.inp(' ')
#obj.inp('a')
#obj.inp('#')

[(2, 'i love leetcode'), (2, 'ironman'), (3, 'island'), (5, 'i love you')]

### Alien dictionary

In [37]:
from collections import defaultdict

class Solution(object):
    def alienOrder(self, words):
        """
        :type words: List[str]
        :rtype: str
        
        Ideas:
        - build directed graph, to top. sort
        - how to build graph?
        - consider every (ordered pair of words)
        - find first uncommon letter, l1, l2
        - put edge l1 -> l2
            
        """
        
        isDAG, G = self.buildGraph(words)
        return self.topSort(G) if isDAG and len(G) > 0 else ''
    
        
    def buildGraph(self,words):
        """
        
        Ideas:
        - consider each ordered pair of words
        - find first index of first letter they do not have in common
        - add letter1 and letter2 to G
        - add edge from letter1 -> letter2
        
        """
        
        isDAG, G = True, {}
        
        #edge case
        if len(words) == 1:
            letters = [l for l in words[0]]
            G = self.addNodesToGraph(G,letters)
            return isDAG,G
        
        #nonedge
        for i in range(len(words)):
            for j in range(i+1,len(words)):
                word1, word2 = words[i], words[j]
                k, G = self.findIndexFirstUncommonLetter(word1,word2,G)
                if k != -1:                                #condition for words not being the same
                    letter1, letter2 = word1[k], word2[k]
                    print('edge to add = {} -> {}'.format(letter1,letter2))
                    isDAG, G = self.addEdgeToGraph(G,letter1,letter2)
                    print('isDAG = {}'.format(isDAG))
                    if not isDAG: return isDAG,G
        return isDAG,G
    

    def addNodesToGraph(self,G,letters):
        for letter in letters:
            if letter not in G:
                G[letter] = set()
        return G
    
    def addEdgeToGraph(self,G,letter1,letter2):
        """
        Need to check for cycle here too
        (require the word graph to be acyclic)
        
        Add letter1 -> letter2
        But check if 
        
        """
        
        isDAG = True
        if letter1 in G[letter2]:
            isDAG = False
        G[letter1].add(letter2)  #represented as sets so I don't have to worry about duplicates
        return isDAG,G
    
    
    def findIndexFirstUncommonLetter(self,word1,word2,G):
        """
        
        word1 = 'wrt'
        word2 = 'wrx' --> k = 2
        
        word1 = 'wrt'
        word2 = 'ert' --> k = 0
        
        word1 = 'wr'
        word2 = 'wrx' --> k = -1
        
        Q: can I assume all same length? No!
        
        """

        k = 0
        flag = False
        word1, word2 = [l1 for l1 in word1], [l2 for l2 in word2]  #make queues here
        while word1 and word2:
            l1, l2 = word1.pop(0), word2.pop(0)
            G = self.addNodesToGraph(G,[l1,l2])
            if l1 != l2:
                flag = True
                break
            k += 1
                        
        #now add remaining letters to graph
        #if word1 is not same length as word2
        remainingLetters = word1 + word2
        G = self.addNodesToGraph(G,remainingLetters)
        
        #If break flag went off, then k was a 'normal' value
        if flag:
            return k, G
        
        #Otherwise
        #terminal cases: (i) 'abc', 'abc'  --> k = -1
        #                (ii) 'abc', 'ab' --> k = -1
        else:
            k = -1
            return k,G
    

    def topSort(self,G):
        stack = []
        visited = set()

        def helper(G,node):
            visited.add(node)
            for n in G[node]:
                if n not in visited:
                    helper(G,n)
            stack.insert(0,node)

        for n in list(G.keys()):
            if n not in visited:
                helper(G,n)

        return "".join(stack)
                    

words = ["xyz"]
sol = Solution()
#G = sol.buildGraph(words)
#G
sol.alienOrder(words)

'zyx'

### Decode ways a given signal can be encoded

In [146]:
from functools import lru_cache

#@lru_cache(maxsize=None)
def f(s):
    """
    
    BRUTE FORCE:
    - enumerate all 'permutations'
    - count allowable ones
    
    Time: O(n!), Space: O(1)
    
    BETTER:
    - recursion with caching
    
    Time: O(n), Space: O(n)
    
    """
            
    if len(s) == 0:
        yield 'flag'       
    if len(s) == 1:
        yield [s]
    else:
        for i in range(1,len(s)+1):  # s = 'abc'. first: 'a, bc', last: '', 'abc'
            left, right = s[:i], s[i:]
            for p in f(right):
                if p != 'flag':
                    yield [left] + p
                else:
                    yield [left]  
                    
s = 'abc'
list(f(s))

[['a', 'b', 'c'], ['a', 'bc'], ['ab', 'c'], ['abc']]

In [190]:
def dp(s):
    """
    
    Simpler case: pretend there is map for all two digits numbers: 1->A, ... 99->Z!
    
    P[i] = number of ways s_1, ... s_{i} can be sorted
        
    Boundary conditions
    P[0] = 0
    P[1] = 1
    P[2] = 2
    P[3] = ?
    
    Ex: s = '1215'
    
    P[1] = [1]
    P[2] = [[1,2],[12]]
    
    P[3] = [[1,2,1],[12,1]]
    P[4] = [[1,2,1,5],[12,1,5], |||, [1,2,15], [12,15] ]  
    
    - third number can only pair with neighbour?
    
    if 0 < s[i] <= 0:
        P[i] = P[i-1]
    if 0 < s[i-1:i+1] <= 26:
        P[i] = P[i-2] + 1 ?
        
    """
    
    #edge case
    if s[0] == '0':a
        return 0
    if len(s) == 0:
        return 0
    
    #BCs
    P = [0]*(len(s)+1)
    P[0] = 1 
    
    for i in range(1,len(s)+1):
        j = i-1
        
        #single digits
        if 0 < int(s[j]) <= 9:
            P[i] = P[i-1]
        
        #double digits
        #print('i, s[i-1:i+1] = {},{}'.format(i, s[j-1:j+1]))
        if s[j-1:j+1] and 10 <= int(s[j-1:j+1]) <= 26:
            P[i] += P[i-2]
            
    return P

s = '1'
dp(s)

i, s[i-1:i+1] = 1,1


[1, 1]

### The skyline problem

Not yet finished.

In [ ]:
from heapq import heappush

def f(buildings):
    
    curr_building = 0
    events = make_events(building)
    heap = []
    
    for (x,H,i) in events[1]:
        
        #if building in heap, then have to jump --> KP
        if (H,i) in heap:
            heap.del((H,i))
            (H_new, new_building) = heap[0]
            sols.append[(x,H_new)]
            curr_building
        
        #if building not in heap, then add to heap
        
        #and check if new jump
    
def make_events(buildings)

### Subsequences with m odd number

In [24]:
def f(arr,m):
    """
    
    Ex: 
    arr = [2,5,6,9], m = 2
    sol = [2,5,6,9], [5,6,9]
    
    
    IDEAS:
    - use two pointer solution
    - move r until num odd = m, record as r0
    - move r until arr[r+1] = odd,
    - record l0
    - move l until num odd < m 
    - now count all subarrays with starts in [l0,l] and ends in [r0,r]
    - repeat
    
    Ex:    0 1 2 3 4 5 6
    arr = [2,2,5,6,9,2,11], m = 2
               l0
                   l
                       r0
                        r  
                 
        sols = 0
        num_odd = 2
    
    """
    
    n = len(arr)
    l, l0 = 0, 0
    r, r0 = 0, 0
    num_odd, count = 0, 0
    while r < n:
        
        #move right until num of odd > m
        while num_odd < m and r < n:
            if arr[r] % 2 != 0:
                num_odd += 1
            r += 1
        
        #record r0, and move r until NEXT element is odd
        r0 = r-1
        while r + 1 < n and arr[r+1] % 2 == 0:
            r += 1
        
        #record l0
        l0 = l
        
        #move l until num odd < m
        while num_odd >= m and l < r:
            l += 1
            if arr[l] % 2 != 0:
                num_odd -= 1
                
        #now record count
        temp = (l-l0+1)*(r-r0+1)
        count += temp
        print('l0, l, r0, r, tmep = {}, {}, {}, {}, {}'.format(l0,l,r0,r,temp))
        
    return count

m = 2
#arr = [2,5,6,9]
arr = [2, 2, 5, 6, 9, 2, 11]
f(arr,m)

l0, l, r0, r, tmep = 0, 2, 4, 5, 6
l0, l, r0, r, tmep = 2, 4, 6, 7, 6


12

### Microsoft: card game

https://leetcode.com/discuss/interview-question/482921/Microsoft-or-Phone-or-Design-Card-Game

In [28]:
from random import shuffle

class Deck:
    def __init__(self):
        self.deck  = self._make_deck()
        
    def _make_deck(self):
        """
        Suits: Hearts, Diamond, Spades clubs
        
        7H -- seven of hearts
        
        A,2,3,...10,J,Q,K
        
        """
        
        nums = ['A'] + [str(i) for i in range(2,11)] + ['J','Q','K']
        suits = ['H','D','S','C']
        cards = [ n + s for n in nums for s in suits ]
        return cards
    
    def shuffle(self):
        shuffle(self.deck)
        return self.deck
    
    
deck = Deck()

['10S',
 'AD',
 'JS',
 '5H',
 '7H',
 '3H',
 'JD',
 '2D',
 '10H',
 '10D',
 '3D',
 'QS',
 'KC',
 'AH',
 '6H',
 '6S',
 'QH',
 '2H',
 'JC',
 '9C',
 '4C',
 '9D',
 'QD',
 'KD',
 '3S',
 '4S',
 'KS',
 '6D',
 'JH',
 'KH',
 '7C',
 '8H',
 '8S',
 '5C',
 '2C',
 '5D',
 '3C',
 'QC',
 '10C',
 '9S',
 '2S',
 'AC',
 'AS',
 '5S',
 '9H',
 '7D',
 '4D',
 '8D',
 '6C',
 '4H',
 '8C',
 '7S']

### Microsoft: Pop Balloons

https://leetcode.com/discuss/interview-question/462752/Microsoft-or-Phone-or-Pop-Matrix-Balloons

1. Pop balloons doing DFS
2. Do gravity -- ?


Ex:  3 2 3
     0 3 1
     0 2 2
     1 1 1
     3 3 2
     0 1 0
     
 BRUTE FORCE:
 - find zero (start from left hand corner, that way dont have to worry about zeros)
 - move to top vertically; while board[i-1][j] != 0 and i-1 >= 0
 
 Time: O(n^2) to search for zeros + bubble up time = O(n^2)?
 
 BETTER:
 - 

### Microsoft: Stacking boxes

https://leetcode.com/discuss/interview-question/386397/Microsoft-or-Phone-Screen-or-Stacking-Boxes

In [ ]:
def f(arr):
    """
    
    sol =  [[2,3],[5,4],[6,4],[6,7]]
              i  
    
    IDEAS:
    - overlapping intervals?
    - sort by width, then iterate and 'squash'
    - do i have to worry about order?
    - iterate over i
    - move j until x conditon not met (and check y condition)

    Time: O(n log n), Space: O(1)
    

    """

### Leetcode: lowest common ancestor in binary tree

In [1]:
def LCA(root,p,q):
    """
    
    Ideas:
    - BFS for both p and q, then compare paths?
    - is path unique? yes
    """
    
    queue = [(root,[root])]
    path_p, path_q = [], []
    while queue and (not path_p or not path_q):
        node,path = queue.pop(0)
        
        #check paths
        if node.val == p:
            path_p = path + [node]
        if node.val == q:
            path_q = path + [node]
        
        #add neighbours
        if node.left:
            queue.append((node.left,path+[node.left]))
        if node.right:
            queue.append((node.right,path+[node.right]))

    #compare paths
    print('path_p = {}'.format([x.val for x in path_p]))
    print('path_q = {}'.format(path_q))
    while path_p[0].val == path_q[0].val:
        sol = path_p.pop(0)
        path_q.pop(0)
    return sol.val
        
tree = BST(None)   #from data-structures file
tree.insertVals([3,5,1,6,2,0,8,7,4])
root = tree.root
LCA(root,4,8)

NameError: name 'BST' is not defined

### Num coins

You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1


Input: coins = [1, 2, 5], amount = 11
Output: 3 
Explanation: 11 = 5 + 5 + 1

Input: coins = [2], amount = 3
Output: -1

322. Coin Change

In [54]:
coins = [2]
amount = 10

D = [0]*(amount+1)
for i in range(1,len(D)):
    temp = [D[i-j] for j in coins if (i-j)>=0 and D[i-j] != -1]
    if temp:
        D[i] = 1 + min(temp)
    else:
        D[i] = -1
D

[0, -1, 1, -1, 2, -1, 3, -1, 4, -1, 5]

### Find median of two sorted arrays -- not yet finished

In [61]:
import numpy as np

x = [1,10,11,12]
y = [3,4,13]

sorted(x+y)

[1, 3, 4, 10, 11, 12, 13]

In [ ]:
def median(x,y):
    """
    
    Define i as x_i, s.t x_{i+1} in right half of joined array
           j as y_j, s.t. y_{j+1} in right half of joined array
    
    Two conditions:
    
    (i) i + j  = (m+n) // 2 
    (ii) if x_i > y_j
            y_{j} <= x_i <= y_{j+1}
         else:
            x_{i} <= y_j <= x_{i+1}
            
    So can eliminate j, and do a binary search?
           
    """

    

In [95]:
def binarySearch(arr,target):
    l,r = 0, len(arr)-1
    while l <= r:
        mid = (r+l) // 2
        if arr[mid] < target:
            l = mid + 1
        elif arr[mid] > target:
            r = mid - 1
        else:
            return mid
    return False


def medianSortedArrays(x,y):
    n,m = len(x), len(y)
    l, r = 0, len(x) - 1
    while l <= r:
        i = (r+l) // 2
        j = ((m+n-1) // 2) - i
        
        print('i,j,r,l = {}, {}, {}, {}'.format(i,j,l,r))
        
        #first make sure j is legal
        if j > m-1:
            l = i + 1
        elif j < 0:
            r = i - 1
            
        #then do check
        elif j+1 > m-1 or x[i] > y[j+1]:
            l = i+1
        elif x[i] < y[j]:
            r = i-1
        elif y[j] <= x[i] <= y[j+1]:
            return x[i]

    return x[l]

x = [1,10,11,12,15,16]
y = [3,4,13]


print('true sol = {}'.format(f(x,y)))
print('guess sol = {}'.format(medianSortedArrays(x,y)))

true sol = 11
i,j,r,l = 2, 2, 0, 5
i,j,r,l = 4, 0, 3, 5
i,j,r,l = 5, -1, 5, 5
guess sol = 16


### Transformation exists

(x,y) --> (x, y+x) or (x+y, y)

Can you get from (sx,sy) to (tx,ty)   where x,y, in Z^2?

Sol

In [11]:
def f(sx,sy,tx,ty):
    """
    
    Idea: 
    - start at (tx,ty)
    - if there is a path to (tx,ty), then path to (tx-ty, ty) AND path to (tx,tx-ty)
    - so, only need to explore (tx-ty,ty) OR (tx,ty-tx)
    - recursing, we see we only need to explore a verical jump to tx == sx
      or a horizontal jump to the line x = sy
    - in other words, need to check if ty is a multiple of (tx-sx)? and tx is a multiple of (ty-sy)?
    
    """
    
    #edge case
    if sx > tx or sy > ty:
        return False

    #Get as close to sx and sy as possible
    #Idea: clean this up using remainder
    while tx > sx and ty > sy:
        if tx > ty:
            k = (tx-sx) // ty
            tx -= k*ty
        else:
            k = (ty-sy) // tx
            ty -= k*tx

        #If not a multiple, then no solution
        if k < 1:
            return False

    print('sx,sy = {},{}'.format(sx,sy))
    print('tx,ty = {}, {}'.format(tx,ty))
    print('x,y = {},{}'.format(tx,ty))

    #Now check if hit the target
    #Could hit it exactly, up y be a multiple of it
    if (tx-sx) % ty == 0 and (ty-sy) % tx == 0:
        return True
    return False

### Same as above,

But neet to count number of paths. Then do DP and count

Time: O(n^2)
Space: O(n^2)

n = max(tx-sx,ty-sy)

Do bottom up, have to scan over entire plane
Do top down, only need to scan 'patched' plane -- better

In [10]:
(sx,sy) = (1,1)
(tx,ty) = (16,3)

#Set up
M = {}
M[(sx,sy)] = 1

#Iterate
for i in range(sx,tx+1):
    for j in range(sy, ty+1):
        temp1 = M[(i,j-i)] if (i,j-i) in M else 0
        temp2 = M[(i-j,j)] if (i-j,j) in M else 0
        if temp1 > 0 or temp2 > 0:
            M[(i,j)] = temp1 + temp2
            

M[(tx,ty)] if (tx,ty) in M else 0 

1

In [33]:
import networkx as nx

def dfs(G):
    
    visited = set()
    stack = []
    
    def helper(G,node):
        for n in G[node]:
            if n not in visited:
                visited.add(n)
                helper(G,n)
        stack.insert(0,node)

    
    for n in G.nodes():
        if n not in visited:
            helper(G,n)
            
    return stack

G = nx.DiGraph() 
G.add_edge(5, 2); 
G.add_edge(5, 0); 
G.add_edge(4, 0); 
G.add_edge(4, 1); 
G.add_edge(2, 3); 
G.add_edge(3, 1);
dfs(G)

[4, 5, 0, 2, 3, 1]

### Merge intervals

Not finished

In [60]:
class Solution(object):
    def insert(self, intervals, newInterval):
        """
        :type intervals: List[List[int]]
        :type newInterval: List[int]
        :rtype: List[List[int]]
        
        Ideas:
        - check if overlap
        - merge, check for merge cascase
        
        find overlap
        - scan through list until found a merge
        - then do merge
        - then check for merge cascade
        
        - find position in list (check if newIx > I[i].x for first time)
        - then check if is it overlaps with left neighbour
        - if yes, merge.
        - this could trigger a merge right cascade
        - while I[i] overlaps with I[i+1]
        - merge 
        
        - if there's no overlaps, just insert
        - So need an overLap check, and a merge
        - Need to do the merge in place...
        
        newI = [4,5]
        I = [[1,3], [6,9]]
                   i
                   
        Plan:
        - find position to insert
        - insert
        - while overlap(left) or overlap(right)
        - merge
                
        """
        
 
        i, intervals = self.insertToList(intervals, newInterval)
            
            
        #Now, while there is overlap, merge left, then merge right
        left = intervals[i-1] if i-1 >= 0 else None
        new = intervals[i]
        right = intervals[i+1] if i+1 < len(intervals) else None
        
        print('i = {}: l, n, r = {}, {}, {}'.format(i,left,new,right))
        
        while self.overlap(left,new) or self.overlap(new,right):
            
            #Do merge
            if self.overlap(left,new):
                mergedInterval = self.merge(left,new)
                intervals[i-1] = mergedInterval
                del intervals[i]  #must be cleaner way to do this
            else:
                mergedInterval = self.merge(new,right)
                print('i, mergedInterval, intervals = {}, {}, {}'.format(i,mergedInterval,intervals))
                intervals[i] = mergedInterval
                del intervals[i+1]
                
            #Update
            left = intervals[i-1] if i-1 >= 0 else None
            new = intervals[i]
            right = intervals[i+1] if i+1 < len(intervals) else None
                
        return intervals
    
    
    def insertToList(self,intervals, newInterval):
        """ Inserts into the merged list
            and respects order. i is position of 
        """
        for i in range(len(intervals)):
            if newInterval[0] > intervals[i][0]:
                break
        intervals = intervals[:i+1] + [newInterval] + intervals[i+1:]
        return i, intervals
        
    
        
    def overlap(self,i1,i2):
        """
        
        Assume i1 starts before i2.
        Then there is an overlap if i2 begins
        beefore i1 ends

        """
        
        if i1 is None or i2 is None:
            return False

        i1,i2 = sorted([i1,i2])  #i1 starts before i2
        start1, end1 = i1
        start2, end2 = i2
        return start2 <= end1
    
    
    def merge(self,i1,i2):
        """
        
        i1 = [start1,end1]
        i2 = [start2,end2]
        
        i_new = [min(s1,s2), max(e1,e2)]
        
        """
        
        start1, end1 = i1
        start2, end2 = i2
        return [min(start1,start2), max(end1,end2)]
        
sol = Solution()
intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]]
newInterval = [4,8]
sol.insert(intervals,newInterval)

i = 0: l, n, r = None, [1, 2], [4, 8]


[[1, 2], [4, 8], [3, 5], [6, 7], [8, 10], [12, 16]]

### Smallest palindrome

In [5]:
def isPalindrome(s):
    return s == s[::-1]

s = "aabba"

sol = s[::-1] + s

while s[0] == s[-1]:
    s = s[1:-1]
s

'abb'

In [3]:


isPalindrome(sol)

True